Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности
$$\frac{d}{dx}[k(x)\frac{du}{dx}] - q(x)u = -f(x)$$
в 11 равноудаленных точках отрезка $[0, 1]$ с относительной точностью $0.0001$.
Отладку программы произвести на модельной задаче с постоянными коэффициентами.

In [17]:
from numpy import sin, cos, exp, sqrt, linspace, round, array, max, abs
from pandas import DataFrame

Условие. Задача 4.

Краевые условия:
$$u(0) = 1; u(1) = 0$$
\begin{equation*}
 \begin{cases}
    u(x_0 - 0) = u(x_0 + 0) \\
    k(x_0 - 0) u_{x}(x_0 - 0) = k(x_0 + 0) u_{x}(x_0 + 0) 
 \end{cases}
\end{equation*}
$$x > x_0 = 1/\sqrt{2} : k(x) = e^{-x}; q(x) = x^2; f(x) = 1$$
$$x < x_0: k(x) = 1; q(x) = e^{-x^2}; f(x) = cos(x)$$

Модельная задача:
$$x_0 = 1/\sqrt{2} : k(x) = k(x_0); q(x) = q(x_0); f(x) = f(x_0)$$

In [18]:
def ka(x):
    return exp(-x)


# k for i < la
def kb(x):
    return 1


# q for i < la
def qa(x):
    return x**2


# q for i > lb
def qb(x):
    return exp(-(x)**2)


# f for i > lb
def fa(x):
    return 1

# f for i < la
def fb(x):
    return cos(x)

In [19]:
#условия
accur = 10**(-4)

u0 = 1
u1 = 0

x0 = 2**(-0.5)



Аналитическое решение модельной задачи

In [20]:
def cs(l1, l2):
    a11 = exp(-l1*x0)-exp(l1*x0)
    a12 = exp(l2*(2-x0))-exp(l2*x0)
    a21 = ka(x0)*l1*(exp(l1*x0)+exp(-l1*x0))
    a22 = kb(x0)*l2*(exp(l2*(2-x0))+exp(l2*x0))
    ma = fa(x0)/qa(x0)
    mb = fb(x0)/qb(x0)
    b1 = mb-ma+(ma-u0)*exp(l1*x0)-(mb-u1)*exp(l2*(1-x0))
    b2 = ka(x0)*l1*(u0-ma)*exp(l1*x0)+kb(x0)*l2*(u1-mb)*exp(l2*(1-x0))
    c1 = (((u0-ma)*a11-b1)*a22-((u0-ma)*a21-b2)*a12)/(a11*a22-a12*a21)
    c2 = (b1*a22-b2*a12)/(a11*a22-a12*a21)
    c3 = (b2*a11-b1*a21)/(a11*a22-a12*a21)
    c4 = (u1-mb)*exp(l2)-c3*exp(2*l2)
    return c1, c2, c3, c4

def model_u(x, l_1, l_2, c_1, c_2, c_3, c_4):
    if x < x0:
        c1 = c_1
        c2 = c_2
        m = fa(x0)/qa(x0)
        l1 = l_1
    else:
        c1 = c_3
        c2 = c_4
        m = fb(x0)/qb(x0)
        l1 = l_2
    return c1 * exp(l1 * x) + c2 * exp(-l1 * x) + m


Прогонка


In [21]:
def counter_sweep_method(a, b, c, d, la, lb):
    n = len(a)
    alpha = [0] * (n-1)
    alpha[1] = -a[1]/b[1]
    alpha[-1] = -c[-1]/b[-1]
    beta = [0] * (n-1)
    beta[1] = (d[1]-c[1]*u0)/b[1]
    beta[-1] = (d[-1]-c[-1]*u1)/b[-1]
    x = [u0] * n
    x[-1] = u1
    for i in range(2, la):
        alpha[i] = (-a[i]/(c[i]*alpha[i-1]+b[i]))
        beta[i] = ((d[i]-c[i]*beta[i-1])/(b[i]+c[i]*alpha[i-1]))
    for i in reversed(range(lb+1, n-2)):
        alpha[i] = -c[i]/(b[i]+a[i]*alpha[i+1])
        beta[i] = (d[i]-a[i]*beta[i+1])/(b[i]+a[i]*alpha[i+1])
    x[lb] = x[la] = (ka(x0)*beta[la-1]+kb(x0)*beta[lb+1])/(ka(x0)*(1-alpha[la-1])+kb(x0)*(1-alpha[lb+1]))
    for i in reversed(range(1, la)):
        x[i] = alpha[i]*x[i+1]+beta[i]
    for i in range(lb+1, n-1):
        x[i] = alpha[i]*x[i-1]+beta[i]
    return x

def u_model_comp(n):
    a, b, c, d = init_const_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

Численное решение модельной задачи с заданной степенью точности(Метод встречных прогонок)

In [22]:
def init_const_abcd(n):
    a, b, c, d = [], [], [], []
    for i in range(n):
        a.append(ka(x0) if i < la else kb(x0))
        b.append(-2 * ka(x0) - qa(x0) * h ** 2 if i < la else -2 * kb(x0) - qb(x0) * h ** 2)
        c.append(ka(x0) if i < la else kb(x0))
        d.append(-fa(x0) * h ** 2 if i < la else -fb(x0) * h ** 2)
    return a, b, c, d

def u_model_comp(n):
    a, b, c, d = init_const_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

def u_var_comp(n):
    a, b, c, d = init_variable_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]


Численное решение задачи с переменными коэффициентами 

In [23]:

def init_variable_abcd(n):
    a, b, c, d = [], [], [], []
    for i in range(n):
        a.append(ka(h * (i + 1. / 2)) if i < la else kb(h * (i + 1. / 2)))
        b.append(-(ka(h * (i + 1. / 2)) + ka(h * (i - 1. / 2)) + qa(h * i) * h ** 2) if i < la else -(
        kb(h * (i + 1. / 2)) + kb(h * (i - 1. / 2)) + qb(h * i) * h ** 2))
        c.append(ka(h * (i - 1. / 2)) if i < la else kb(h * (i - 1. / 2)))
        d.append(-fa(h * i) * h ** 2 if i < la else -fb(h * i) * h ** 2)
    return a, b, c, d

def u_var_comp(n):
    a, b, c, d = init_variable_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

In [24]:


#разделения отрезка на n частей
L = 11#менять на сдаче

x = round(linspace(0.0, 1.0, L), 7)

h = 1 / L

la = int(x0 / h)
lb = int(round(x0 / h))

rep_x = linspace(0.0, 1.0, 11)
l1 = sqrt(qa(x0)/ka(x0))
l2 = sqrt(qb(x0)/kb(x0))
c1, c2, c3, c4 = cs(l1, l2)

u_analytical = array([model_u(i, l1, l2, c1, c2, c3, c4) for i in rep_x])
u_sweep = array(u_model_comp(L))
u_sweep_tmp = array(u_var_comp(L))

dat = {
    'x' : rep_x,
    'Analytical solve' : u_analytical,
    'Sweep solve' : u_sweep,
    'Error' : u_analytical - u_sweep,
    'Max error' : max(abs(u_analytical - u_sweep)),
    'Sweep tmp solve' : u_sweep_tmp
}
df = DataFrame(data=dat)
df

,x,Analytical solve,Sweep solve,Error,Max error,Sweep tmp solve
0,0.0,1.000000e+00,1.000000,0.000000e+00,0.105485,1.000000
1,0.1,9.322537e-01,0.911935,2.031879e-02,0.105485,0.940724
2,0.2,8.536708e-01,0.814751,3.891954e-02,0.105485,0.866408
3,0.3,7.634536e-01,0.707634,5.581917e-02,0.105485,0.774944
4,0.4,6.606864e-01,0.589687,7.099977e-02,0.105485,0.664069
5,0.5,5.443264e-01,0.459920,8.440680e-02,0.105485,0.531293
6,0.6,4.131926e-01,0.317246,9.594684e-02,0.105485,0.373751
7,0.7,2.659540e-01,0.160469,1.054847e-01,0.105485,0.187953
8,0.8,1.802384e-01,0.160469,1.976919e-02,0.105485,0.187953
9,0.9,9.363825e-02,0.083168,1.047056e-02,0.105485,0.096341
